In [109]:
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [2]:
def decomp1d(n, N, block_index):
    """_summary_

    Args:
        n (int): array lenght
        N (int): number of blocks to split into
        block_index (int): index of block

    Returns:
        s, e (tuple): start and end indices of block within unlbokced array
    """
    remainder = n % N
    base = n // N

    if (block_index < remainder):
        s = block_index * (base+1)
        e = s + base+1
        return s, e 
    else:
        s = (remainder * (base+1)) + (max(block_index-remainder, 0) * base)
        e = s + base
        return s, e 
   
def decomp_matrix(A, M, N):
    """Docomposes an input matrix A into a block matrix of shape (M,N) in a load balanced way.

    Args:
        A (matrix): input matrix to decompose into a matrix of M*N blocks
        M (int): number of block rows
        N (int): number of block columns

    Returns:
        block_A (block matrix): block matrix version of A 
    """
    
    block_A = [[0]*N for _ in range(M)]
    m, n = len(A), len(A[0])
    
    for i in range(M):
        s1, e1 = decomp1d(m, M, i)
        for j in range(N):
            s2, e2 = decomp1d(n, N, j)
            sub_M = e1 - s1
            sub_N = e2 - s2
            block_A[i][j] = np.zeros((sub_M, sub_N))
            for a in range(sub_M):
                for b in range(sub_N):
                    block_A[i][j][a][b] = A[s1 + a][s2 + b]

    return block_A

def comp_matrix(m, n, block_mat):
    """Composes a matrix of shape (m,n) from a block matrix.

    Args:
        m (int): number of rows of output matrix
        n (int): number of columns of output matrix
        block_mat (block matrix): a matrix of matrices in each entry

    Returns:
        B (matrix) : matrix corresponding to the block matrix inputed of shape (m, n)
    """
    B = np.zeros((m, n))
    block_rows = len(block_mat)
    block_cols = len(block_mat[0])
    
    row_step = 0
    col_step = 0
    sub_rows = 0
    sub_cols = 0
    
    for i in range(block_rows):
        for j in range(block_cols):
            block = np.array(block_mat[i][j])
            
            sub_rows, sub_cols = len(block), len(block[0])
            for a in range(sub_rows):
                for b in range(sub_cols):
                    B[row_step + a][col_step + b] = block[a][b]
                    
            col_step += sub_cols
        row_step += sub_rows
        col_step = 0
            
    return B

In [107]:
def TSQR(A, max_block_rows=4): # NOT COMPLETE QR FACTORISATION
    """An implementation of TSQR that divides an input matrix 
    up into four blocks of rows (using row sub-indexing) and computes the
    QR-factorisation in the way shown in the lectures on communication-avoiding 
    factorisations. 

    Args:
        A (matrix): matrix to find QR factorisation of
        max_block_rows (int): number of block rows to initially decompose the matrix into.

    Returns:
        Q, R (tuple of matrices):
    """
    
    m, n = len(A), len(A[0])
    Q = np.eye(m)
    R = decomp_matrix(A, max_block_rows, 1)
    
    print(R)
    print(f"len(R), len(R[0]) = {len(R), len(R[0])}")
    
    rows_num = max_block_rows
    while rows_num > 1:
            
        Q_temp = [[None]*rows_num for _ in range(rows_num)]
        R_temp = [[None]*1 for _ in range(rows_num)]
        print(f"R_temp shape ={len(R_temp), len(R_temp[0])}")
        
        for i in range(rows_num):
            print(f"i={i}")
            #print(f"rows num = {rows_num}")
            print(f"R[0][i].shape ={R[i][0].shape}")
            
            Qi, Ri = np.linalg.qr(R[i][0] , mode="reduced")
            
            R_temp[i][0] = Ri
            Q_temp[i][i] = Qi
            
        for i in range(rows_num):
            for j in range(rows_num):
                if i!=j:
                    m0 = len(Q_temp[i][i])
                    Q_temp[i][j] = np.zeros((m0, n))
           
        #m1 = m if rows_num == max_block_rows else rows_num*n*2
        if rows_num == max_block_rows:
            m1 = m
        else:
            m1 = n*rows_num*2

        n1 = rows_num * n
        #print(f"m1, n1 = {m1, n1}")
        
        Q = Q @ comp_matrix(m1, n1, Q_temp)
        
        rows_num = rows_num // 2
        
        #R = [[None]*rows_num]
        R = [[None]*1 for _ in range(rows_num)]
        for i in range(rows_num):
            R1 = R_temp[i*2][0]
            R2 = R_temp[i*2 + 1][0]
            m0 = len(R1) + len(R2)
            n0 = n
            
            R[i][0] = comp_matrix(m0, n0, [[R1], [R2]])
        
        print("pass")
        
        
    Q_temp, R_temp = np.linalg.qr(R[0][0])
    Q = Q @ Q_temp
    R = R_temp
    
    return Q, R

In [108]:
#n = np.random.randint(low=10, high=100)
#m = n * np.random.randint(low=2, high=10)
m, n = 10, 4
A = np.random.rand(m, n)

print(f"A.shape={A.shape}")
npQ, npR = np.linalg.qr(A)
Q, R = TSQR(A, 4)

print(f"MSE(Q - npQ) = {np.mean(np.square(Q - npQ))}")
print(f"MSE(R - npR) = {np.mean(np.square(R - npR))}")

A.shape=(10, 4)
[[array([[0.71574009, 0.73086412, 0.3574231 , 0.10583098],
       [0.07779409, 0.11589652, 0.16885848, 0.36509561],
       [0.85704878, 0.60439246, 0.58820404, 0.53883495]])], [array([[0.3449336 , 0.13019225, 0.84585419, 0.11440597],
       [0.7978075 , 0.72666302, 0.77598232, 0.67595433],
       [0.14296701, 0.3591303 , 0.64240173, 0.03437303]])], [array([[0.52490953, 0.78330159, 0.20717251, 0.1059814 ],
       [0.47221052, 0.33098613, 0.25348166, 0.16753487]])], [array([[0.88549397, 0.96374636, 0.73887208, 0.21374499],
       [0.2141926 , 0.25929533, 0.65251614, 0.93162299]])]]
len(R), len(R[0]) = (4, 1)
R_temp shape =(4, 1)
i=0
R[0][i].shape =(3, 4)
i=1
R[0][i].shape =(3, 4)
i=2
R[0][i].shape =(2, 4)
i=3
R[0][i].shape =(2, 4)
pass
R_temp shape =(2, 1)
i=0
R[0][i].shape =(6, 4)
i=1
R[0][i].shape =(4, 4)
pass
MSE(Q - npQ) = 0.14988663745092304
MSE(R - npR) = 0.30743152143126157
